## Duck Typing, Goose Typing, Static Typing and Static duck typing

There are 4 ways to define user intefaces in Python

![](./static/flpy_1301.png)

- Duck Typing (oldest one)
- Goose Typing using abstract base classes (ABCs) (Python 2.6+)
- Static Typing ([PEP 484](https://peps.python.org/pep-0484/) - Python 3.5+)
- Static Duck Typing by subclasses of typing.Protocol (Python 3.8+)

### Duck Typing

In [1]:
numbers = list(range(1, 6))

# Checking length
print(f"{len(numbers) = }")

# Indexing
print(f"{numbers[0] = }")
print(f"{numbers[-2] = }")

# Slicing
print(f"{numbers[1:4] = }")

# Membership testing
print(f"{3 in numbers = }")
print()

# Looping
for number in numbers:
    print(number, end="|")

len(numbers) = 5
numbers[0] = 1
numbers[-2] = 4
numbers[1:4] = [2, 3, 4]
3 in numbers = True

1|2|3|4|5|

In [2]:
# Custom class sequence using magic methods (a.k.a double-underscore method or dunder method)
# https://docs.python.org/3/glossary.html#term-sequence
class HalfNumbers:
    def __init__(self, start, stop):
        self.start = start
        self.stop = stop
        self.numbers = list(range(self.start, self.stop))
        
    def __len__(self):
        return len(self.numbers)
    
    def __getitem__(self, index):
        if isinstance(index, slice):
            return [number / 2 for number in self.numbers[index]]
        else:
            return self.numbers[index] / 2

In [3]:
half_numbers = HalfNumbers(1, 6)

# Checking length
print(f"{len(half_numbers) = }")

# Indexing
print(f"{half_numbers[0] = }")
print(f"{half_numbers[-2] = }")

# Slicing
print(f"{half_numbers[1:4] = }")

# Membership testing
print(f"{1.5 in half_numbers = }")
print()

# Looping
for half_number in half_numbers:
    print(half_number, end="|")

len(half_numbers) = 5
half_numbers[0] = 0.5
half_numbers[-2] = 2.0
half_numbers[1:4] = [1.0, 1.5, 2.0]
1.5 in half_numbers = True

0.5|1.0|1.5|2.0|2.5|

### Goose Typing and ABCs (Abstract base classes)

![](./static/flpy_1304.png)

In [4]:
from collections.abc import Sized

class InvalidSizedClass(Sized):
    def __init__(self):
        print('initiating')
        
invalid_sized = InvalidSizedClass()

TypeError: Can't instantiate abstract class InvalidSizedClass with abstract methods __len__

In [5]:
class ValidSizedClass(Sized):
    def __len__(self):
        return 3

valid_sized = ValidSizedClass()
print(f"{len(valid_sized) = }")
print(f"{isinstance(valid_sized, Sized) = }")

len(valid_sized) = 3
isinstance(valid_sized, Sized) = True


### Static Typing

In [6]:
%load_ext save_and_exec_magic

In [7]:
%%save_and_run_magic mypy
def scale(scalar, vector):
    return [scalar * num for num in vector]


print(f"{scale(2, [1, 2, 3, 4]) = }\n")
print(f"{scale(2, [1.0, 2.0, 3.0, 4.0]) = }\n")
print(f"{scale(2, [1, 2.0, 3, 4.0]) = }\n")
print(f"{scale(2, (1, 2, 3, 4)) = }\n")
print(f"{scale(2, {1:1, 2:2, 3:3, 4:4}) = }\n")

In [8]:
%%save_and_run_magic mypy
from typing import List


Vector = List[float]

def scale(scalar: float, vector: Vector) -> Vector:
    return [scalar * num for num in vector]


print(f"{scale(2, [1, 2, 3, 4]) = }\n")
print(f"{scale(2, [1.0, 2.0, 3.0, 4.0]) = }\n")
print(f"{scale(2, [1, 2.0, 3, 4.0]) = }\n")
print(f"{scale(2, (1, 2, 3, 4)) = }\n")
print(f"{scale(2, {1:1, 2:2, 3:3, 4:4}) = }\n")

### Static duck typing

In [9]:
%%save_and_run_magic mypy
from decimal import Decimal

pi = Decimal(22) / Decimal(7)

def double(num):
    return num * 2

print(f"{double(4) = }")
print(f"{double('D') = }")
print(f"{double([1, 2, 3, 4]) = }")
print(f"{double(pi) = }")
print(f"{double({1:1, 2:2}) = }")

In [10]:
print(f"{'__mul__' in dir(int) = }")
print(f"{'__mul__' in dir(str) = }")
print(f"{'__mul__' in dir(list) = }")
print(f"{'__mul__' in dir(dict) = }")

'__mul__' in dir(int) = True
'__mul__' in dir(str) = True
'__mul__' in dir(list) = True
'__mul__' in dir(dict) = False


In [11]:
%%save_and_run_magic mypy
from decimal import Decimal
from typing import Union, List

pi = Decimal(22) / Decimal(7)

def double(num: Union[int, str, List[int], Decimal]):
    return num * 2

print(f"{double(4) = }")
print(f"{double('D') = }")
print(f"{double([1, 2, 3, 4]) = }")
print(f"{double(pi) = }")
print(f"{double({1:1, 2:2}) = }")

In [12]:
print(f"{'__mul__' in dir(tuple) = }")

'__mul__' in dir(tuple) = True


In [13]:
%%save_and_run_magic mypy
from decimal import Decimal
from typing import TypeVar, Protocol

T = TypeVar('T')

pi = Decimal(22) / Decimal(7)

class Repeatable(Protocol):
    def __mul__(self: T, repeat_count: int) -> T: ...

RT = TypeVar('RT', bound=Repeatable)

def double(x: RT) -> RT:
    return x * 2

print(f"{double(4) = }")
print(f"{double('D') = }")
print(f"{double([1, 2, 3, 4]) = }")
print(f"{double(pi) = }")
print(f"{double((1, 2, 3)) = }")
print(f"{double({1:1, 2:2}) = }")